In [15]:
#Librerías
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import numpy as np
import gc
from datetime import datetime
import os

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib') 
pd.options.mode.chained_assignment = None

In [16]:
sellin = pd.read_csv('/home/agu/projects/notebooks/maestria/labo3/dataset/sell-in.txt.gz', compression='gzip', sep=r'\s+', engine='python')
productos = pd.read_csv('/home/agu/projects/notebooks/maestria/labo3/dataset/tb_productos.txt', sep='\t')
stocks = pd.read_csv('/home/agu/projects/notebooks/maestria/labo3/dataset/tb_stocks.txt', sep='\t')
productos_a_predecir= pd.read_csv("/home/agu/projects/notebooks/maestria/labo3/dataset/product_id_apredecir201912.txt", sep='\t')


In [17]:
#sellin.shape #2945818
sellin = sellin[sellin['product_id'].isin(productos_a_predecir['product_id'])]

In [18]:
df_prod_periodo = sellin.groupby(['product_id','periodo']).agg(tn=('tn', 'sum'))

In [19]:
df_prod_periodo['clase'] = df_prod_periodo.shift(-2)
#df_prod_periodo = df_prod_periodo.fillna(0)

In [20]:
df_prod_periodo.head(2)#

tn     clase
product_id periodo                   
20001      201701   934.772  1303.358
           201702   798.016  1069.961

In [7]:
#df_prod_periodo.reset_index()

df = df_prod_periodo.copy()
df = df.sort_index(level=['product_id','periodo'])
# 2) Forzar NaN en 201911 y 201912
periodos_sin_target = [201911, 201912]

mask = df.index.get_level_values('periodo').isin(periodos_sin_target)
df.loc[mask, 'clase'] = np.nan

# 1) Creamos los 11 lags (tn_1 = mes anterior, tn_2 = dos meses atrás, … tn_11)
for i in range(1, 12):
    df[f'tn_{i}'] = (
        df
        .groupby(level='product_id')['tn']
        .shift(i)
    )

df['media_tn'] = (
    df
      .groupby(level='product_id')['tn']    # todas las filas de cada ID
      .transform('mean')                    # promedio y lo replica fila a fila
)

#cada fila tiene:
#  - tn_producto_total  (mes t)
#  - tn_1 … tn_11       (meses t−1 … t−11, con NaN si no hay historia)
#  - tn_mean_11         (media de los meses anteriores disponibles)
#df=df.fillna(0)
df.head(37)


tn     clase      tn_1      tn_2      tn_3  \
product_id periodo                                                     
20001      201701    934.772  1303.358       NaN       NaN       NaN   
           201702    798.016  1069.961   934.772       NaN       NaN   
           201703   1303.358  1502.201   798.016   934.772       NaN   
           201704   1069.961  1520.065  1303.358   798.016   934.772   
           201705   1502.201  1030.674  1069.961  1303.358   798.016   
           201706   1520.065  1267.395  1502.201  1069.961  1303.358   
           201707   1030.674  1316.946  1520.065  1502.201  1069.961   
           201708   1267.395  1439.756  1030.674  1520.065  1502.201   
           201709   1316.946  1580.474  1267.395  1030.674  1520.065   
           201710   1439.756  1049.389  1316.946  1267.395  1030.674   
           201711   1580.474  1169.075  1439.756  1316.946  1267.395   
           201712   1049.389  1043.765  1580.474  1439.756  1316.946   
           201801   1169.075  1856.835  1049.389  1580.474  1439.756   
           201802   1043.765  1251.285  1169.075  1049.389  1580.474   
           201803   1856.835  1293.898  1043.765  1169.075  1049.389   
           201804   1251.285  1150.792  1856.835  1043.765  1169.075   
           201805   1293.898  1470.410  1251.285  1856.835  1043.765   
           201806   1150.792  1800.962  1293.898  1251.285  1856.835   
           201807   1470.410  1438.675  1150.792  1293.898  1251.285   
           201808   1800.962  2295.198  1470.410  1150.792  1293.898   
           201809   1438.675  1813.015  1800.962  1470.410  1150.792   
           201810   2295.198  1486.687  1438.675  1800.962  1470.410   
           201811   1813.015  1275.774  2295.198  1438.675  1800.962   
           201812   1486.687  1259.094  1813.015  2295.198  1438.675   
           201901   1275.774  1470.657  1486.687  1813.015  2295.198   
           201902   1259.094  1647.638  1275.774  1486.687  1813.015   
           201903   1470.657  1629.782  1259.094  1275.774  1486.687   
           201904   1647.638  1109.938  1470.657  1259.094  1275.774   
           201905   1629.782  1678.993  1647.638  1470.657  1259.094   
           201906   1109.938  1261.345  1629.782  1647.638  1470.657   
           201907   1678.993  1660.006  1109.938  1629.782  1647.638   
           201908   1261.345  1561.506  1678.993  1109.938  1629.782   
           201909   1660.006  1397.372  1261.345  1678.993  1109.938   
           201910   1561.506  1504.689  1660.006  1261.345  1678.993   
           201911   1397.372       NaN  1561.506  1660.006  1261.345   
           201912   1504.689       NaN  1397.372  1561.506  1660.006   
20002      201701    550.157   834.735       NaN       NaN       NaN   

                        tn_4      tn_5      tn_6      tn_7      tn_8  \
product_id periodo                                                     
20001      201701        NaN       NaN       NaN       NaN       NaN   
           201702        NaN       NaN       NaN       NaN       NaN   
           201703        NaN       NaN       NaN       NaN       NaN   
           201704        NaN       NaN       NaN       NaN       NaN   
           201705    934.772       NaN       NaN       NaN       NaN   
           201706    798.016   934.772       NaN       NaN       NaN   
           201707   1303.358   798.016   934.772       NaN       NaN   
           201708   1069.961  1303.358   798.016   934.772       NaN   
           201709   1502.201  1069.961  1303.358   798.016   934.772   
           201710   1520.065  1502.201  1069.961  1303.358   798.016   
           201711   1030.674  1520.065  1502.201  1069.961  1303.358   
           201712   1267.395  1030.674  1520.065  1502.201  1069.961   
           201801   1316.946  1267.395  1030.674  1520.065  1502.201   
           201802   1439.756  1316.946  1267.395  1030.674  1520.065   
           201803   1580.474  1439.756  1316.946  1267.395  1030.

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df_feat = df.copy().fillna(0)

#df_feat.rename(columns={'tn_producto_total':'tn'}, inplace=True)

# 2) Filtramos solo periodo de entrenamiento = 201812
dataset_training = df_feat.xs(201812, level='periodo').dropna(subset=['clase'])
#   .xs extrae nivel periodo=201812, luego eliminamos si no hay target

# 3) Definimos X y y
feature_cols = ['tn'] + [f'tn_{i}' for i in range(1,12)] ## solo los lags entran a la regresion lineal. 
X_train = dataset_training[feature_cols]
y_train = dataset_training['clase']

# 4) Entrenamos regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# 5) Métrica de entrenamiento (opcional)
y_pred = model.predict(X_train)
mse    = mean_squared_error(y_train, y_pred)
print(f"MSE entrenamiento (201812→201902): {mse:.3f}")

# Ya tienes `model` listo para predecir en periodo 201901
# (en un df con las mismas columnas X de fecha 201901).


MSE entrenamiento (201812→201902): 246.862


In [9]:
dataset_training.shape

(655, 14)

In [10]:
COL   = 'tn_11'

# --- 1.  Máscara fila‐a‐fila ----------------------------------------------
mask = df_feat[COL].notna() & (df_feat[COL] != 0)   # True si tn_11 válido

# --- 2.  Filtrar DataFrame (opcional) --------------------------------------
df_tn11_ok = df_feat[mask]          # solo filas con tn_11 ≠ 0/NaN

# --- 3.  Lista única de product_id que tienen todos los lags... -----------------------------------------
ids_tn11_ok = (
    df_feat.index.get_level_values('product_id')[mask]
            .unique()
            .tolist()
)

print(f"{len(ids_tn11_ok)} productos cumplen la condición de tener todos los tn_lags")


all_ids = df_feat.index.get_level_values('product_id').unique()

# -----------------------------------------------------------------
# 2)  Diferencia de conjuntos  →  los que NO están en ids_tn11_ok
# -----------------------------------------------------------------
ids_tn11_bad = sorted(set(all_ids) - set(ids_tn11_ok))

print(f"{len(ids_tn11_bad)} productos NO cumplen la condición de tener todos los lags.")

656 productos cumplen la condición de tener todos los tn_lags
124 productos NO cumplen la condición de tener todos los lags.


In [22]:
magicos = [
 20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
 20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046,
 20049, 20051, 20052, 20053, 20055, 20008, 20001, 20017,
 20086, 20180, 20193, 20320, 20532, 20612, 20637, 20807, 20838
]


# Extraemos sólo periodo 201812 y sólo los “mágicos”
train_df = (
    df_feat
    .loc[(magicos, 201812), :]      # máscara multi‐índice
    .dropna(subset=['clase'])      # eliminamos filas con clase NaN (no deberían quedar)
)

# Definimos X e y
feature_cols = ['tn'] + [f'tn_{i}' for i in range(1,12)]
X_train_magicos = train_df[feature_cols]
y_train_magicos = train_df['clase']
# pids   = train_df['product_id']


# Entrenamiento de regresión lineal “out-of-the-box”
model_magicos = LinearRegression()
model_magicos.fit(X_train_magicos, y_train_magicos)

# Métrica en el mismo conjunto (opcional, solo 33 puntos)
y_pred_magicos = model_magicos.predict(X_train_magicos)
mse_magicos   = mean_squared_error(y_train_magicos, y_pred_magicos)
print(f"MSE en los 33 “mágicos”: {mse_magicos:.3f}")

# pred_df = pd.DataFrame({
#         'product_id'      : pids,
#         'prediccion_tn+2' : y_pred_magicos           # toneladas esperadas en 201902
# }).set_index('product_id')

# print(pred_df.head())

MSE en los 33 “mágicos”: 954.939


In [23]:
# df_feat.loc[pd.IndexSlice[ids_tn11_bad, :], :].reset_index()

In [24]:
# -------------------- 5. PREDICCIÓN PARA LOS COMPLETOS -----------------------
feature_cols = ['tn'] + [f'tn_{i}' for i in range(1,12)]
snap_ok = (
    df_feat
      .loc[pd.IndexSlice[ids_tn11_ok, 201912], :]   # multi-índice (product_id, periodo)
      .reset_index()                                  # columnas normales
)
snap_incompleto = (
    df_feat
      .loc[pd.IndexSlice[ids_tn11_bad, 201912], :]   # multi-índice (product_id, periodo)
      .reset_index()                                  # columnas normales
)

X_complete = snap_ok[feature_cols]

In [25]:
snap_ok.shape,snap_incompleto.shape

((656, 16), (124, 16))

In [26]:
#pip install dtaidistance

In [27]:
print(x_candidato.shape, x_m.shape)


(12,) (12,)


In [28]:
# X_complete = snap_ok[feature_cols]
X_complete_no_completo = snap_incompleto[feature_cols]

yhat = model_magicos.predict(X_complete)

pred_ok = pd.DataFrame({
    'product_id'     : snap_ok['product_id'],
    'tn'      : yhat
})

# -------------------- 6. FILL PARA LOS INCOMPLETOS ---------------------------
#  ↳ aquí uso la media del target de entrenamiento; ajusta si quieres otra lógica
#fallback = y_train_magicos.mean()          # ≈ promedio histórico de los mágicos

pred_na = pd.DataFrame({
    'product_id'     : snap_incompleto['product_id'],
    'tn'      : snap_incompleto['media_tn']
})

# -------------------- 7. UNIMOS Y GUARDAMOS ----------------------------------
pred_final = (
    pd.concat([pred_ok, pred_na], ignore_index=True)
      .sort_values('product_id')
)




25785.020397754266

In [29]:

pred_ok['tn'].shape


(656,)

In [30]:
pred_final.shape


(780, 2)

In [31]:
pred_ok['tn'].sum()

25785.020397754266

In [32]:
pred_final.to_csv("predicciones_201902_reg_lineal2.csv", index=False)


In [ ]:
####### HASTA ACA 
######
